<a href="https://colab.research.google.com/github/JuniorTorresMTJ/Bootcamp_Data_Science_Aplicada_Alura/blob/main/notebook/BoletoFraud_Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [10]:
color =['#87B738',
        '#283610',
        '#ABCC73']

In [11]:
df_boleto = pd.read_csv("https://raw.githubusercontent.com/JuniorTorresMTJ/FraudBoleto/main/Data/dataset_case_boleto.csv", sep=";")

In [12]:
df_boleto['max_valor_boleto'] = df_boleto['max_valor_boleto'].str.replace(',', '.')
df_boleto['avg_valor_boleto'] = df_boleto['max_valor_boleto'].str.replace(',', '.')
df_boleto['total_valor_boleto'] = df_boleto['max_valor_boleto'].str.replace(',', '.')
df_boleto['valor_boleto_stdv'] = df_boleto['max_valor_boleto'].str.replace(',', '.')
df_boleto['mes_ref'] = pd.to_datetime(df_boleto['mes_ref'], format='%d/%m/%Y')
df_boleto['max_valor_boleto'] = df_boleto['max_valor_boleto'].astype(float)
df_boleto['avg_valor_boleto'] = df_boleto['avg_valor_boleto'].astype(float)
df_boleto['total_valor_boleto'] = df_boleto['total_valor_boleto'].astype(float)
df_boleto['valor_boleto_stdv'] = df_boleto['valor_boleto_stdv'].astype(float)
df_boleto

,mes_ref,company_id,tipo_doc,max_valor_boleto,avg_valor_boleto,total_valor_boleto,valor_boleto_stdv,qtd_boleto_pago,qtd_boleto_total,qtd_boleto_estorno,qnt_cc_total,tempo_credenciamento,conta_bnk_repetida,fraude
0,2020-03-01,5e74d202d1498c5bdf8aafe3,cnpj,6000.0,6000.0,6000.0,6000.0,10,13,0,35,11,0,0
1,2020-03-01,5e78dafeb5ac867c7e85eb5e,cnpj,1800.0,1800.0,1800.0,1800.0,0,2,0,0,11,0,0
2,2020-03-01,5e73e21359193c2f123c1076,cnpj,159.9,159.9,159.9,159.9,0,2,0,14,11,0,0
3,2020-04-01,5e7262c49d55ea5dbea59d57,cnpj,600.0,600.0,600.0,600.0,0,3,0,18,11,0,0
4,2020-04-01,5e610d9f66945c0f82dd357f,cnpj,2941.0,2941.0,2941.0,2941.0,2,5,0,65,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9272,2020-11-01,5fa6013aec647c00160a173b,cpf,50.0,50.0,50.0,50.0,0,1,0,0,2,0,1
9273,2020-11-01,5fc3ed1659018400162a83ad,cpf,10.0,10.0,10.0,10.0,1,1,0,0,3,0,0
9274,2020-10-01,5f6f796ccff5532d540c6f58,cpf,160.0,160.0,160.0,160.0,0,1,0,0,5,0,0
9275,2020-12-01,5ebd605d4c4cb062ab6b3c66,cnpj,600.0,600.0,600.0,600.0,0,1,0,0,9,0,0


In [13]:
total_boletos = df_boleto.groupby(['company_id']).size().reset_index(name='total_boletos')
total_boletos

,company_id,total_boletos
0,5e0eb0115e364024155a696c,1
1,5e0f70e84331ba63bb6a3eb0,9
2,5e1485a98e158b0d6cfc4df9,1
3,5e149751d9d52a271763c0e1,1
4,5e14ac4c6204025f8ec345a3,4
...,...,...
4955,60442a0bb22d6100118bc21a,1
4956,60444661b22d6100118bc276,1
4957,60444c6505b7f90019aa3e20,1
4958,60445669a8b6c8001136e544,1


In [14]:
fraud_last_month = df_boleto.sort_values(by=['company_id', 'mes_ref'])
fraud_last_month['last_month'] = fraud_last_month['mes_ref'] + pd.DateOffset(months=-1)
fraud_last_month['fraud_last_month'] = fraud_last_month.groupby('company_id')['fraude'].shift(1)
fraud_last_month['fraud_last_month'].fillna(0, inplace=True)
fraud_last_month['fraud_last_month'] = fraud_last_month['fraud_last_month'].astype(int)
fraud_last_month

,mes_ref,company_id,tipo_doc,max_valor_boleto,avg_valor_boleto,total_valor_boleto,valor_boleto_stdv,qtd_boleto_pago,qtd_boleto_total,qtd_boleto_estorno,qnt_cc_total,tempo_credenciamento,conta_bnk_repetida,fraude,last_month,fraud_last_month
166,2020-01-01,5e0eb0115e364024155a696c,cpf,597.95,597.95,597.95,597.95,0,4,0,3,14,0,0,2019-12-01,0
1228,2020-01-01,5e0f70e84331ba63bb6a3eb0,cnpj,576.00,576.00,576.00,576.00,1,3,0,2,14,0,0,2019-12-01,0
4767,2020-06-01,5e0f70e84331ba63bb6a3eb0,cnpj,227.39,227.39,227.39,227.39,1,2,0,14,14,0,0,2020-05-01,0
1943,2020-08-01,5e0f70e84331ba63bb6a3eb0,cnpj,368.88,368.88,368.88,368.88,2,2,0,12,14,0,0,2020-07-01,0
8004,2020-09-01,5e0f70e84331ba63bb6a3eb0,cnpj,3184.00,3184.00,3184.00,3184.00,3,3,0,5,14,0,0,2020-08-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5943,2021-03-01,60442a0bb22d6100118bc21a,cnpj,10020.00,10020.00,10020.00,10020.00,0,2,0,0,0,0,0,2021-02-01,0
2343,2021-03-01,60444661b22d6100118bc276,cnpj,12090.00,12090.00,12090.00,12090.00,0,2,0,0,0,0,0,2021-02-01,0
3078,2021-03-01,60444c6505b7f90019aa3e20,cnpj,26541.71,26541.71,26541.71,26541.71,0,2,0,0,0,0,0,2021-02-01,0
3545,2021-03-01,60445669a8b6c8001136e544,cnpj,17681.37,17681.37,17681.37,17681.37,0,1,0,0,0,0,0,2021-02-01,0


In [15]:
total_fraudes_por_empresa = df_boleto.groupby('company_id')['fraude'].sum().reset_index()
total_fraudes_por_empresa.sort_values('fraude', ascending = False)

,company_id,fraude
1339,5ecd307ba13ae25ec966033e,7
1735,5efb636407ab9b56e30b38ed,6
2196,5f6026109537112db6b611a8,5
2441,5f8faa1998c7b700111d1865,5
2264,5f6a15f6dfe01c6043588f22,5
...,...,...
1601,5ee76f72684921056d5df5b0,0
1600,5ee768b8389a0a4d6f57aad1,0
1599,5ee7677d2ba8850506c4c0df,0
1598,5ee766b3113f12056ea3d661,0


In [16]:
fraud_last_2_month = df_boleto.sort_values(by=['company_id', 'mes_ref'])
fraud_last_2_month['last_2_month'] = fraud_last_2_month['mes_ref'] + pd.DateOffset(months=-2)
fraud_last_2_month['fraud_last_2_month'] = fraud_last_2_month.groupby('company_id')['fraude'].shift(2)
fraud_last_2_month['fraud_last_2_month'].fillna(0, inplace=True)
fraud_last_2_month['fraud_last_2_month'] = fraud_last_2_month['fraud_last_2_month'].astype(int)
fraud_last_2_month

,mes_ref,company_id,tipo_doc,max_valor_boleto,avg_valor_boleto,total_valor_boleto,valor_boleto_stdv,qtd_boleto_pago,qtd_boleto_total,qtd_boleto_estorno,qnt_cc_total,tempo_credenciamento,conta_bnk_repetida,fraude,last_2_month,fraud_last_2_month
166,2020-01-01,5e0eb0115e364024155a696c,cpf,597.95,597.95,597.95,597.95,0,4,0,3,14,0,0,2019-11-01,0
1228,2020-01-01,5e0f70e84331ba63bb6a3eb0,cnpj,576.00,576.00,576.00,576.00,1,3,0,2,14,0,0,2019-11-01,0
4767,2020-06-01,5e0f70e84331ba63bb6a3eb0,cnpj,227.39,227.39,227.39,227.39,1,2,0,14,14,0,0,2020-04-01,0
1943,2020-08-01,5e0f70e84331ba63bb6a3eb0,cnpj,368.88,368.88,368.88,368.88,2,2,0,12,14,0,0,2020-06-01,0
8004,2020-09-01,5e0f70e84331ba63bb6a3eb0,cnpj,3184.00,3184.00,3184.00,3184.00,3,3,0,5,14,0,0,2020-07-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5943,2021-03-01,60442a0bb22d6100118bc21a,cnpj,10020.00,10020.00,10020.00,10020.00,0,2,0,0,0,0,0,2021-01-01,0
2343,2021-03-01,60444661b22d6100118bc276,cnpj,12090.00,12090.00,12090.00,12090.00,0,2,0,0,0,0,0,2021-01-01,0
3078,2021-03-01,60444c6505b7f90019aa3e20,cnpj,26541.71,26541.71,26541.71,26541.71,0,2,0,0,0,0,0,2021-01-01,0
3545,2021-03-01,60445669a8b6c8001136e544,cnpj,17681.37,17681.37,17681.37,17681.37,0,1,0,0,0,0,0,2021-01-01,0


In [17]:
fraud_last_3_month = df_boleto.sort_values(by=['company_id', 'mes_ref'])
fraud_last_3_month['last_3_month'] = fraud_last_3_month['mes_ref'] + pd.DateOffset(months=-3)
fraud_last_3_month['fraud_last_3_month'] = fraud_last_3_month.groupby('company_id')['fraude'].shift(3)
fraud_last_3_month['fraud_last_3_month'].fillna(0, inplace=True)
fraud_last_3_month['fraud_last_3_month'] = fraud_last_3_month['fraud_last_3_month'].astype(int)
fraud_last_3_month

,mes_ref,company_id,tipo_doc,max_valor_boleto,avg_valor_boleto,total_valor_boleto,valor_boleto_stdv,qtd_boleto_pago,qtd_boleto_total,qtd_boleto_estorno,qnt_cc_total,tempo_credenciamento,conta_bnk_repetida,fraude,last_3_month,fraud_last_3_month
166,2020-01-01,5e0eb0115e364024155a696c,cpf,597.95,597.95,597.95,597.95,0,4,0,3,14,0,0,2019-10-01,0
1228,2020-01-01,5e0f70e84331ba63bb6a3eb0,cnpj,576.00,576.00,576.00,576.00,1,3,0,2,14,0,0,2019-10-01,0
4767,2020-06-01,5e0f70e84331ba63bb6a3eb0,cnpj,227.39,227.39,227.39,227.39,1,2,0,14,14,0,0,2020-03-01,0
1943,2020-08-01,5e0f70e84331ba63bb6a3eb0,cnpj,368.88,368.88,368.88,368.88,2,2,0,12,14,0,0,2020-05-01,0
8004,2020-09-01,5e0f70e84331ba63bb6a3eb0,cnpj,3184.00,3184.00,3184.00,3184.00,3,3,0,5,14,0,0,2020-06-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5943,2021-03-01,60442a0bb22d6100118bc21a,cnpj,10020.00,10020.00,10020.00,10020.00,0,2,0,0,0,0,0,2020-12-01,0
2343,2021-03-01,60444661b22d6100118bc276,cnpj,12090.00,12090.00,12090.00,12090.00,0,2,0,0,0,0,0,2020-12-01,0
3078,2021-03-01,60444c6505b7f90019aa3e20,cnpj,26541.71,26541.71,26541.71,26541.71,0,2,0,0,0,0,0,2020-12-01,0
3545,2021-03-01,60445669a8b6c8001136e544,cnpj,17681.37,17681.37,17681.37,17681.37,0,1,0,0,0,0,0,2020-12-01,0


In [18]:
fraud_last_6_month = df_boleto.sort_values(by=['company_id', 'mes_ref'])
fraud_last_6_month['last_6_month'] = fraud_last_6_month['mes_ref'] + pd.DateOffset(months=-6)
fraud_last_6_month['fraud_last_6_month'] = fraud_last_6_month.groupby('company_id')['fraude'].shift(6)
fraud_last_6_month['fraud_last_6_month'].fillna(0, inplace=True)
fraud_last_6_month['fraud_last_6_month'] = fraud_last_6_month['fraud_last_6_month'].astype(int)
fraud_last_6_month

,mes_ref,company_id,tipo_doc,max_valor_boleto,avg_valor_boleto,total_valor_boleto,valor_boleto_stdv,qtd_boleto_pago,qtd_boleto_total,qtd_boleto_estorno,qnt_cc_total,tempo_credenciamento,conta_bnk_repetida,fraude,last_6_month,fraud_last_6_month
166,2020-01-01,5e0eb0115e364024155a696c,cpf,597.95,597.95,597.95,597.95,0,4,0,3,14,0,0,2019-07-01,0
1228,2020-01-01,5e0f70e84331ba63bb6a3eb0,cnpj,576.00,576.00,576.00,576.00,1,3,0,2,14,0,0,2019-07-01,0
4767,2020-06-01,5e0f70e84331ba63bb6a3eb0,cnpj,227.39,227.39,227.39,227.39,1,2,0,14,14,0,0,2019-12-01,0
1943,2020-08-01,5e0f70e84331ba63bb6a3eb0,cnpj,368.88,368.88,368.88,368.88,2,2,0,12,14,0,0,2020-02-01,0
8004,2020-09-01,5e0f70e84331ba63bb6a3eb0,cnpj,3184.00,3184.00,3184.00,3184.00,3,3,0,5,14,0,0,2020-03-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5943,2021-03-01,60442a0bb22d6100118bc21a,cnpj,10020.00,10020.00,10020.00,10020.00,0,2,0,0,0,0,0,2020-09-01,0
2343,2021-03-01,60444661b22d6100118bc276,cnpj,12090.00,12090.00,12090.00,12090.00,0,2,0,0,0,0,0,2020-09-01,0
3078,2021-03-01,60444c6505b7f90019aa3e20,cnpj,26541.71,26541.71,26541.71,26541.71,0,2,0,0,0,0,0,2020-09-01,0
3545,2021-03-01,60445669a8b6c8001136e544,cnpj,17681.37,17681.37,17681.37,17681.37,0,1,0,0,0,0,0,2020-09-01,0


In [19]:
fraud_last_12_month = df_boleto.sort_values(by=['company_id', 'mes_ref'])
fraud_last_12_month['last_12_month'] = fraud_last_12_month['mes_ref'] + pd.DateOffset(months=-12)
fraud_last_12_month['fraud_last_12_month'] = fraud_last_12_month.groupby('company_id')['fraude'].shift(12)
fraud_last_12_month['fraud_last_12_month'].fillna(0, inplace=True)
fraud_last_12_month['fraud_last_12_month'] = fraud_last_12_month['fraud_last_12_month'].astype(int)
fraud_last_12_month

,mes_ref,company_id,tipo_doc,max_valor_boleto,avg_valor_boleto,total_valor_boleto,valor_boleto_stdv,qtd_boleto_pago,qtd_boleto_total,qtd_boleto_estorno,qnt_cc_total,tempo_credenciamento,conta_bnk_repetida,fraude,last_12_month,fraud_last_12_month
166,2020-01-01,5e0eb0115e364024155a696c,cpf,597.95,597.95,597.95,597.95,0,4,0,3,14,0,0,2019-01-01,0
1228,2020-01-01,5e0f70e84331ba63bb6a3eb0,cnpj,576.00,576.00,576.00,576.00,1,3,0,2,14,0,0,2019-01-01,0
4767,2020-06-01,5e0f70e84331ba63bb6a3eb0,cnpj,227.39,227.39,227.39,227.39,1,2,0,14,14,0,0,2019-06-01,0
1943,2020-08-01,5e0f70e84331ba63bb6a3eb0,cnpj,368.88,368.88,368.88,368.88,2,2,0,12,14,0,0,2019-08-01,0
8004,2020-09-01,5e0f70e84331ba63bb6a3eb0,cnpj,3184.00,3184.00,3184.00,3184.00,3,3,0,5,14,0,0,2019-09-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5943,2021-03-01,60442a0bb22d6100118bc21a,cnpj,10020.00,10020.00,10020.00,10020.00,0,2,0,0,0,0,0,2020-03-01,0
2343,2021-03-01,60444661b22d6100118bc276,cnpj,12090.00,12090.00,12090.00,12090.00,0,2,0,0,0,0,0,2020-03-01,0
3078,2021-03-01,60444c6505b7f90019aa3e20,cnpj,26541.71,26541.71,26541.71,26541.71,0,2,0,0,0,0,0,2020-03-01,0
3545,2021-03-01,60445669a8b6c8001136e544,cnpj,17681.37,17681.37,17681.37,17681.37,0,1,0,0,0,0,0,2020-03-01,0


In [20]:

# Taxa de estorno para boletos pagos: qtd_boleto_estorno / qtd_boleto_pago
df_boleto['taxa_estorno_pago'] = df_boleto.apply(
    lambda row: row['qtd_boleto_estorno'] / row['qtd_boleto_pago'] if row['qtd_boleto_pago'] != 0 else (0 if row['qtd_boleto_estorno'] == 0 else np.nan),
    axis=1
)


# Taxa de pagamento de boletos: qtd_boleto_pago / qtd_boleto_total
df_boleto['taxa_pagamento'] = df_boleto['qtd_boleto_pago'] / df_boleto['qtd_boleto_total']

# Valor médio de boletos pagos: total_valor_boleto / qtd_boleto_pago
df_boleto['valor_medio_pago'] = df_boleto.apply(
    lambda row: row['total_valor_boleto'] / row['qtd_boleto_pago'] if row['qtd_boleto_pago'] != 0 else 0,
    axis=1
)

# Valor médio de boletos estornados: total_valor_boleto / qtd_boleto_estorno
# Evitando divisão por zero
df_boleto['valor_medio_estorno'] = df_boleto.apply(lambda row: row['total_valor_boleto'] / row['qtd_boleto_estorno'] if row['qtd_boleto_estorno'] != 0 else 0, axis=1)


df_boleto


,mes_ref,company_id,tipo_doc,max_valor_boleto,avg_valor_boleto,total_valor_boleto,valor_boleto_stdv,qtd_boleto_pago,qtd_boleto_total,qtd_boleto_estorno,qnt_cc_total,tempo_credenciamento,conta_bnk_repetida,fraude,taxa_estorno_pago,taxa_pagamento,valor_medio_pago,valor_medio_estorno
0,2020-03-01,5e74d202d1498c5bdf8aafe3,cnpj,6000.0,6000.0,6000.0,6000.0,10,13,0,35,11,0,0,0.0,0.769231,600.0,0.0
1,2020-03-01,5e78dafeb5ac867c7e85eb5e,cnpj,1800.0,1800.0,1800.0,1800.0,0,2,0,0,11,0,0,0.0,0.000000,0.0,0.0
2,2020-03-01,5e73e21359193c2f123c1076,cnpj,159.9,159.9,159.9,159.9,0,2,0,14,11,0,0,0.0,0.000000,0.0,0.0
3,2020-04-01,5e7262c49d55ea5dbea59d57,cnpj,600.0,600.0,600.0,600.0,0,3,0,18,11,0,0,0.0,0.000000,0.0,0.0
4,2020-04-01,5e610d9f66945c0f82dd357f,cnpj,2941.0,2941.0,2941.0,2941.0,2,5,0,65,12,0,0,0.0,0.400000,1470.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9272,2020-11-01,5fa6013aec647c00160a173b,cpf,50.0,50.0,50.0,50.0,0,1,0,0,2,0,1,0.0,0.000000,0.0,0.0
9273,2020-11-01,5fc3ed1659018400162a83ad,cpf,10.0,10.0,10.0,10.0,1,1,0,0,3,0,0,0.0,1.000000,10.0,0.0
9274,2020-10-01,5f6f796ccff5532d540c6f58,cpf,160.0,160.0,160.0,160.0,0,1,0,0,5,0,0,0.0,0.000000,0.0,0.0
9275,2020-12-01,5ebd605d4c4cb062ab6b3c66,cnpj,600.0,600.0,600.0,600.0,0,1,0,0,9,0,0,0.0,0.000000,0.0,0.0
